##### Copyright © 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Scikit-Learn Model Card Toolkit Demo

## Background
This notebook demonstrates how to generate a model card using the Model Card Toolkit with a scikit-learn model in a Jupyter/Colab environment. You can learn more about model cards at [https://modelcards.withgoogle.com/about](https://modelcards.withgoogle.com/about).

## Setup
We first need to install and import the necessary packages.

### Upgrade Pip and Install Packages

In [ ]:
try:
    import colab
    !pip install --upgrade pip
except:
    pass

In [ ]:
!pip install -U seaborn
!pip install sklearn
!pip install model-card-toolkit

### Did you restart the runtime?
If you are using Google Colab, the first time that you run the cell above, you must restart the runtime (Runtime > Restart runtime ...). This is because of the way that Colab loads packages.

### Import packages
We import necessary packages, including scikit-learn.

In [1]:
from datetime import date
from io import BytesIO
from IPython import display
from model_card_toolkit import ModelCardToolkit
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix

import base64
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import uuid

ModuleNotFoundError: ignored

## Load data

This example uses the Breast Cancer Wisconsin Diagnostic dataset that scikit-learn can load using the [load_breast_cancer()](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) function.

In [ ]:
cancer = load_breast_cancer()

X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = pd.Series(cancer.target)

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

## Plot data

We will create several plots from the data that we will include in the model card.

In [ ]:
# Utility function that will export a plot to a base-64 encoded string that the model card will accept.

def plot_to_str():
    img = BytesIO()
    plt.savefig(img, format='png')
    return base64.encodebytes(img.getvalue()).decode('utf-8')

In [ ]:
# Plot the mean radius feature for both the train and test sets

sns.displot(x=X_train['mean radius'], hue=y_train)
mean_radius_train = plot_to_str()

sns.displot(x=X_test['mean radius'], hue=y_test)
mean_radius_test = plot_to_str()

In [ ]:
# Plot the mean texture feature for both the train and test sets

sns.displot(x=X_train['mean texture'], hue=y_train)
mean_texture_train = plot_to_str()

sns.displot(x=X_test['mean texture'], hue=y_test)
mean_texture_test = plot_to_str()

## Train model

In [ ]:
# Create a classifier and fit the training data

clf = GradientBoostingClassifier().fit(X_train, y_train)

## Evaluate model

In [ ]:
# Plot a ROC curve

plot_roc_curve(clf, X_test, y_test)
roc_curve = plot_to_str()

In [ ]:
# Plot a confusion matrix

plot_confusion_matrix(clf, X_test, y_test)
confusion_matrix = plot_to_str()

## Create a model card

### Initialize toolkit and model card

In [ ]:
mct = ModelCardToolkit()

model_card = mct.scaffold_assets()

### Annotate information into model card

In [ ]:
model_card.model_details.name = 'Breast Cancer Wisconsin (Diagnostic) Dataset'
model_card.model_details.overview = (
    'This model predicts whether breast cancer is benign or malignant based on '
    'image measurements.')
model_card.model_details.owners = [
    {'name': 'Model Cards Team', 'contact': 'model-cards@google.com'}
]
model_card.model_details.references = [
    'https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)',
    'https://minds.wisconsin.edu/bitstream/handle/1793/59692/TR1131.pdf'
]
model_card.model_details.version.name = str(uuid.uuid4())
model_card.model_details.version.date = str(date.today())

model_card.considerations.ethical_considerations = [{
    'name': ('Manual selection of image sections to digitize could create '
            'selection bias'),
    'mitigation_strategy': 'Automate the selection process'
}]
model_card.considerations.limitations = ['Breast cancer diagnosis']
model_card.considerations.use_cases = ['Breast cancer diagnosis']
model_card.considerations.users = ['Medical professionals', 'ML researchers']


model_card.model_parameters.data.train.graphics.description = (
  f'{len(X_train)} rows with {len(X_train.columns)} features')
model_card.model_parameters.data.train.graphics.collection = [
    {'image': mean_radius_train},
    {'image': mean_texture_train}
]
model_card.model_parameters.data.eval.graphics.description = (
  f'{len(X_test)} rows with {len(X_test.columns)} features')
model_card.model_parameters.data.eval.graphics.collection = [
    {'image': mean_radius_test},
    {'image': mean_texture_test}
]
model_card.quantitative_analysis.graphics.description = (
  'ROC curve and confusion matrix')
model_card.quantitative_analysis.graphics.collection = [
    {'image': roc_curve},
    {'image': confusion_matrix}
]

mct.update_model_card_json(model_card)

## Generate model card

In [ ]:

# Return the model card document as an HTML page
html = mct.export_format()

display.display(display.HTML(html))